# Schema setup for NY taxi dataset

1. Download green and yellow, years 2020-2021
    - `data/taxi_ingest_data/raw/<type>/<filename>`
    - use `download_taxi.sh` script
        - arg1: type
        - arg2: year
    - e.g. `data/taxi_ingest_data/raw/fhv/fhv_tripdata_2020-02.parquet`
    - gzip in place
1. Read raw data file from gzip and set schema
    - read from above
    - set schema
    - repartition
    - write to `data/taxi_ingest_data/staging/<type>/<year>/<month>`

I don't think casting parquet files are a good idea. If we need to recast, cast from `csv`. 

Alternatively read into pandas before writing back into parquet for partitioning

In [3]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pathlib import Path
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 15:06:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [34]:
green_schema = types.StructType([
    types.StructField("VendorID", types.LongType(), True),
    types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("RatecodeID", types.DoubleType(), True),
    types.StructField("PULocationID", types.LongType(), True),
    types.StructField("DOLocationID", types.LongType(), True),
    types.StructField("passenger_count", types.LongType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("ehail_fee", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("payment_type", types.LongType(), True),
    types.StructField("trip_type", types.LongType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

yellow_schema = types.StructType([
    types.StructField("VendorID", types.LongType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.LongType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.DoubleType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.LongType(), True),
    types.StructField("DOLocationID", types.LongType(), True),
    types.StructField("payment_type", types.LongType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

hvfhv_schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('originating_base_num', types.StringType(), True),
    types.StructField('request_datetime', types.TimestampType(), True),
    types.StructField('on_scene_datetime', types.TimestampType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True),
    types.StructField('trip_miles', types.DoubleType(), True),
    types.StructField('trip_time', types.LongType(), True),
    types.StructField('base_passenger_fare', types.DoubleType(), True),
    types.StructField('tolls', types.DoubleType(), True),
    types.StructField('bcf', types.DoubleType(), True),
    types.StructField('sales_tax', types.DoubleType(), True),
    types.StructField('congestion_surcharge', types.DoubleType(), True),
    types.StructField('airport_fee', types.DoubleType(), True),
    types.StructField('tips', types.DoubleType(), True),
    types.StructField('driver_pay', types.DoubleType(), True),
    types.StructField('shared_request_flag', types.StringType(), True),
    types.StructField('shared_match_flag', types.StringType(), True),
    types.StructField('access_a_ride_flag', types.StringType(), True),
    types.StructField('wav_request_flag', types.StringType(), True),
    types.StructField('wav_match_flag', types.StringType(), True)
    ]
)

In [22]:
def cast_schema(spark_client, raw_path: Path, parts_dir: Path, schema: types.StructType, num_parts: int = 4):
    """
    Reads parquet from raw_path, casts the input schema onto it,
    then partitions into output folder
    """
    df = spark_client.read \
        .option('header', 'true') \
        .schema(schema=schema) \
        .parquet(str(raw_path))
    df \
        .repartition(num_parts) \
        .write.parquet(
            str(parts_dir),
            mode='overwrite',
            compression='gzip')


In [25]:
out_dir = Path("../data/taxi_ingest_data/parts/fhvhv/2021/01/")
# if not out_dir.exists():
#     out_dir.mkdir(parents=True)
hv_path = Path("../data/taxi_ingest_data/raw/fhvhv/fhvhv_tripdata_2021-01.parquet")
cast_schema(spark, hv_path, out_dir, hvfhv_schema)


In [26]:
# hvfhv 2021-02
fmonth = "02"
out_dir = Path(f"../data/taxi_ingest_data/parts/fhvhv/2021/{fmonth}/")
# if not out_dir.exists():
#     out_dir.mkdir(parents=True)
hv_path = Path(f"../data/taxi_ingest_data/raw/fhvhv/fhvhv_tripdata_2021-{fmonth}.parquet")
cast_schema(spark, hv_path, out_dir, hvfhv_schema)

In [35]:
taxi_types = ['green', 'yellow']
schemas = [green_schema, yellow_schema]
years = list(range(2020, 2022))
for taxi_type, schema, year in zip(taxi_types, schemas, years):
    for month in range(1, 13):
        raw_path = Path(f"../data/taxi_ingest_data/raw/{taxi_type}/{taxi_type}_tripdata_{year}-{month:02d}.parquet")
        out_dir = Path(f"../data/taxi_ingest_data/parts/{taxi_type}/{year}/{month:02d}/")
        cast_schema(spark, raw_path=raw_path, parts_dir=out_dir, schema=schema)


23/02/20 16:27:47 ERROR Executor: Exception in task 0.0 in stage 14.0 (TID 42)
org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///home/klang/de-zoomcamp/data/taxi_ingest_data/raw/green/green_tripdata_2020-01.parquet. Column: [passenger_count], Expected: bigint, Found: DOUBLE
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(

Py4JJavaError: An error occurred while calling o295.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org